In [0]:
%run ./configs

In [0]:
%run ./vectorSearch

In [0]:
class ProfileIndex:
  def __init__(self):
    self.vs = VectorSearch()

  def get_sync_date(self, index_name):
    sync_date = (
        spark.sql(f"""
            select date_add(last_load_date, 1) as sync_date
            from {ats_configs.jobs_metadate_table_name} 
            where job_name = '{ats_configs.index_sync_job_name}'
            order by last_load_date desc
                  """).first()
                .asDict()["sync_date"]
                .strftime("%Y-%m-%d")
    )

    return sync_date

  def update_metadata(self, sync_date):
    print(f"Updating metadata for {ats_configs.index_sync_job_name}\n")
    spark.sql(
            f"""
            insert into {ats_configs.jobs_metadate_table_name}
            values('{ats_configs.index_sync_job_name}', 
            '{sync_date}', 
            current_timestamp(), 
            'Job Execution Completed.' )""")
    print(f"Updated metadata for {ats_configs.index_sync_job_name}\n")

  def index_profile(self):
    sync_date = self.get_sync_date(ats_configs.vector_index_name)
    print(f"Indexing profiles for {ats_configs.vector_index_name}\n")
    self.vs.sync_index(ats_configs.vector_index_name)
    print(f"Indexed profiles for {ats_configs.vector_index_name}\n")
    self.update_metadata(sync_date)